build_dataset 해당

In [ ]:
from core.macro_classes.macro_class_dataset import MacroAData

macro_agent = MacroAData()
macro_agent.fetch_data()
macro_agent.add_features()
macro_agent.save_csv()
macro_agent.make_close_price()

In [ ]:
from core.macro_classes.macro_class_dataset import MacroAData
# macro_agent = MacroAData()
macro_agent.model_maker()

$$$$   X = macro_sercher(agent, ticker) 역할

In [7]:
from agents.macro_agent import MacroPredictor
from datetime import datetime

macro_sub = MacroPredictor(
    agent_id='MacroSentiAgent',
    base_date=datetime.today(),
    window=40,
    ticker="NVDA"
)
macro_sub.load_assets()               # 모델, 스케일러 등 불러오기


[INFO] 모델 및 스케일러 로드 중...
model_path: models/NVDA_MacroSentiAgent.keras
[OK] 모델 및 스케일러 로드 완료


In [8]:
macro_sub.fetch_macro_data()          # macro_df 불러오기
X_seq, X_scaled = macro_sub.prepare_features()  # 입력 시퀀스 준비

[INFO] MacroSentimentAgent 데이터 수집 중...
1️⃣ Collecting macro features (15 tickers)...


[*********************100%***********************]  15 of 15 completed


[MacroSentimentAgent] Macro data: (43, 91)
   ↳ Downloading NVDA ...


[*********************100%***********************]  1 of 1 completed

[MacroSentimentAgent] Stock data: (42, 9)
[MacroSentimentAgent] Data shape: (42, 99)
[MacroSentimentAgent] Feature engineering complete. Final shape: (42, 170)
[OK] 매크로 데이터 수집 완료: (42, 171)
[INFO] 피처 정리 및 스케일링 중...
[Check] 입력 피처 수: 169 / 스케일러 기준 피처 수: 169
[OK] 스케일링 및 시퀀스 변환 완료


In [ ]:
print(f"X_seq: {X_seq}")
print(f"X_scaled: {X_scaled}")

$$$$   macro_predictor(X) 역할

In [9]:
pred_prices, target = macro_sub.m_predictor(X_seq)

[INFO] 예측 수행 중...
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
NVDA: 마지막 종가=198.95 → 예측 종가=174.85 (예상 수익률 -12.11%)

================= 예측 결과 (표) =================
  Ticker  Last_Close  Predicted_Close  Predicted_Return  Predicted_%  \
0   NVDA  198.951001       174.852971         -0.121125   -12.112545   

   uncertainty  confidence  
0     0.071646   13.957598  

================= 예측 결과 (값) =================
{'NVDA': 174.85297101356718}


In [ ]:
print(f"pred_prices: {pred_prices}")
print(f"target: {target}")

llm 생성 용.. macro_reviewer_draft

In [10]:
 total_json, opinion = macro_sub.macro_reviewer_draft(X_scaled, pred_prices, target)
 

1️⃣ Collecting macro features (15 tickers)...


[*********************100%***********************]  15 of 15 completed


[MacroSentimentAgent] Macro data: (43, 91)
   ↳ Downloading NVDA ...


[*********************100%***********************]  1 of 1 completed


[MacroSentimentAgent] Stock data: (42, 9)
[MacroSentimentAgent] Data shape: (42, 99)
[MacroSentimentAgent] Feature engineering complete. Final shape: (42, 170)

3️⃣ Running GradientAnalyzer for interpretability...
[INFO] Running Gradient × Input + Integrated Gradients analysis...
[DEBUG] Gradient × Input output shape: (1, 42, 169)
[DEBUG] x_input shape before IG: (1, 42, 169)
[DEBUG] interpolated shape: (51, 42, 169)
[INFO] IG–G×I correlation (agreement_ratio): 0.8977
[INFO] Gradient analysis completed successfully.

4️⃣ Generating explanation using LLM...

================= pred_prices:{'NVDA': 174.85297101356718} =================

================= LLM Explanation =================
(1) Feature Trend 분석에서 'Close_GC=F'와 'High_^GSPC' 등 금 선물과 미국 주요 주가지수의 고점 변수가 최근 영향력이 커졌습니다. 이는 금리 변동성과 주식시장 변동성이 금융시장 전반에 미치는 영향이 확대된 거시적 환경을 반영합니다. 반면 일부 통화 및 원자재 변수는 상대적으로 영향력이 약화되었습니다. 이러한 변화는 금과 주가지수의 변동성이 반도체 산업에 대한 투자 심리와 연계되어 예측 방향에 긍정적 신호로 작용할 수 있습니다. (2) Model Consistency 분석에서는 'Close_GC=F'와 'Ope

In [ ]:
import numpy as np

# print("원본 X 평균:", np.mean(X, axis=0)[:10])
print("스케일링 후 X 평균:", np.mean(X_scaled, axis=0)[:10])
print("스케일링 후 X 표준편차:", np.std(X_scaled, axis=0)[:10])
print("pred_prices:", pred_prices)


In [12]:
print(macro_sub.stockdata.MacroSentiAgent["feature_importance"])


{'feature_summary': {'agreement_ratio': 0.8976863906115736, 'gx_importance_top': ['Close_GC=F', 'High_^GSPC', 'Open_^DJI'], 'ig_importance_top': ['Close_GC=F', 'Adj Close_GC=F', 'Open_^IXIC']}, 'importance_dict': {np.str_('Open_CL=F'): 0.00040472199907526374, np.str_('Open_DX-Y.NYB'): 0.00011666095815598965, np.str_('Open_EURUSD=X'): 0.000983817270025611, np.str_('Open_GC=F'): 0.003380795009434223, np.str_('Open_HG=F'): 0.00159740773960948, np.str_('Open_QQQ'): 0.006077147554606199, np.str_('Open_SPY'): 0.006327471695840359, np.str_('Open_USDJPY=X'): 0.001558028976432979, np.str_('Open_^DJI'): 0.006190834566950798, np.str_('Open_^FVX'): 0.0013467223616316915, np.str_('Open_^GSPC'): 0.004437637515366077, np.str_('Open_^IRX'): 0.0012997044250369072, np.str_('Open_^IXIC'): 0.006580164190381765, np.str_('Open_^TNX'): 0.0014000610681250691, np.str_('Open_^VIX'): 0.0006535887368954718, np.str_('High_CL=F'): 0.00024331343593075871, np.str_('High_DX-Y.NYB'): 0.00032507709693163633, np.str_('Hi

In [13]:
print(f"total_json: {total_json}")

total_json: {'agent_id': 'MacroSentiAgent', 'target': Target(next_close=174.85297101356718, uncertainty=0.07164555788040161, confidence=13.957597732543945), 'reason': "(1) Feature Trend 분석에서 'Close_GC=F'와 'High_^GSPC' 등 금 선물과 미국 주요 주가지수의 고점 변수가 최근 영향력이 커졌습니다. 이는 금리 변동성과 주식시장 변동성이 금융시장 전반에 미치는 영향이 확대된 거시적 환경을 반영합니다. 반면 일부 통화 및 원자재 변수는 상대적으로 영향력이 약화되었습니다. 이러한 변화는 금과 주가지수의 변동성이 반도체 산업에 대한 투자 심리와 연계되어 예측 방향에 긍정적 신호로 작용할 수 있습니다. (2) Model Consistency 분석에서는 'Close_GC=F'와 'Open_^DJI' 등 주요 지표가 Integrated Gradients와 Gradient×Input 결과에서 높은 일관성을 보였습니다. 반면 일부 통화 및 변동성 지수는 불일치가 커, 이는 시장 불확실성이나 데이터 잡음, 비선형적 상호작용 때문일 가능성이 있습니다. 일관성 높은 변수들은 모델이 신뢰할 만한 구조적 요인을 잘 반영함을 시사합니다. (3) Sensitivity 분석에서 'Close_GC=F'와 'High_GC=F'가 높은 민감도를 보여 단기 시장 변동성에 민감하며, 이는 포트폴리오 리스크 관리와 단기 트레이딩 전략 수립에 중요한 변수임을 의미합니다. 반면 민감도가 낮은 변수들은 보다 안정적이고 장기적 추세를 반영합니다. (4) Stability 분석에서는 'Close_GC=F'가 낮은 변동성을 보여 안정적 중요도를 유지하는 반면, 일부 변동성 지수는 변동성이 커 시장 국면 전환에 민감함을 나타냅니다. 이는 투자자들이 금과 주요 주가지수에 기반한 장기적 트렌드를 신뢰할 수 있음을 시사합니다. (5) 통합 결론으로, 이번 

In [11]:
print(f"opinion: {opinion}")

opinion: Opinion(agent_id='MacroSentiAgent', target=Target(next_close=174.85297101356718, uncertainty=0.07164555788040161, confidence=13.957597732543945), reason='기술적 분석 측면에서 금 선물(Close_GC=F)의 종가와 고가가 높은 중요도를 보이며, 이는 최근 상승 모멘텀과 변동성 확대를 시사한다. S&P 500 지수(High_^GSPC)와 다우존스 지수(Open_^DJI) 또한 주요 변수로 작용하여, 주요 지수들의 강세가 시장 전반에 긍정적 영향을 미치고 있음을 나타낸다. 거시경제적으로는 금 가격 상승이 인플레이션 우려와 안전자산 선호 심리를 반영하며, 미국 주요 지수의 강세는 경제 회복 기대감과 유동성 공급 지속에 힘입은 것으로 판단된다. 환율 변동성은 상대적으로 낮아 단기 리스크 요인으로 작용하지 않는 반면, 금과 주요 지수의 움직임이 시장 방향성을 주도한다. Gradient × Input과 Integrated Gradients 결과 모두 금 선물과 주요 지수의 종가 및 고가가 높은 중요도를 보이며 일관성을 유지하나, 일부 환율 및 원자재 변수는 순위 차이가 커 민감도와 안정성 측면에서 변동성이 존재한다. 민감도 분석에서 금 선물과 S&P 500 지수 관련 변수들이 높은 민감도를 보여 단기 리스크 요인으로 작용할 수 있으나, 이들의 중요도 안정성은 높아 구조적 상승 추세를 지지한다. 종합하면, 기술적 지표와 거시경제 변수, Gradient 기반 해석 결과가 일관되게 다음 거래일 종가 상승을 시사하며, 이는 인플레이션 헤지 수요와 경제 회복 기대감에 따른 투자 심리 개선에 근거한다. 따라서 다음 거래일 종가는 상승할 가능성이 높으며, 예측값 174.853은 이러한 다각적 분석을 반영한 결과이다.')


reviewer_rebut

In [ ]:
# reviewer_rebut()

[토론 테스트]

In [1]:
from agents.debate_agent import DebateAgent

debate_agent = DebateAgent(1, "NVDA")

In [2]:
debate_agent.run_dataset()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[TechnicalAgent] X_seq: (981, 55, 13), y_seq: (981, 1)
✅ NVDA TechnicalAgent dataset saved to CSV (981 samples, 13 features)
[MacroSentiAgent] X_seq: (1221, 14, 13), y_seq: (1221, 1)
✅ NVDA MacroSentiAgent dataset saved to CSV (1221 samples, 13 features)
[SentimentalAgent] X_seq: (1195, 40, 8), y_seq: (1195, 1)
✅ NVDA SentimentalAgent dataset saved to CSV (1195 samples, 8 features)
✅ NVDA TechnicalAgent dataset saved via technical_data_set
[TRACE B] macro_dataset() start for NVDA


[*********************100%***********************]  15 of 15 completed


[MacroSentimentAgent] Data shape: (1304, 90), Columns: 90
[TRACE A] add_features() for self.data:            Open_CL=F  Open_DX-Y.NYB  Open_EURUSD=X    Open_GC=F  Open_HG=F  \
Date                                                                          
2020-01-01        NaN            NaN       1.122083          NaN        NaN   
2020-01-02  61.599998      96.480003       1.121894  1518.099976     2.8165   
2020-01-03  61.180000      96.790001       1.117081  1530.099976     2.7935   
2020-01-06  63.709999      96.900002       1.116246  1580.000000     2.7780   
2020-01-07  62.910000      96.650002       1.119583  1558.300049     2.8010   
...               ...            ...            ...          ...        ...   
2024-12-24  69.559998     108.160004       1.040583  2613.000000     4.0525   
2024-12-25        NaN            NaN       1.040258          NaN        NaN   
2024-12-26  70.199997     108.169998       1.039955  2628.500000     4.0730   
2024-12-27  69.680000     108.0800

[*********************100%***********************]  101 of 101 completed


저장 완료: (1259, 101) rows
macro: 데이터셋 생성> NVDA
[INFO] Removed all constant Volume columns matching patterns: ['Volume_^FVX', 'Volume_^IRX', 'Volume_^TNX', 'Volume_^VIX', 'Volume_DX-Y.NYB', 'Volume_EURUSD=X', 'Volume_USDJPY=X']
[INFO] Remaining Volume columns: ['Volume_CL=F', 'Volume_GC=F', 'Volume_HG=F', 'Volume_QQQ', 'Volume_SPY', 'Volume_^DJI', 'Volume_^GSPC', 'Volume_^IXIC', 'Volume_CL=F_ret', 'Volume_GC=F_ret', 'Volume_HG=F_ret', 'Volume_QQQ_ret', 'Volume_SPY_ret', 'Volume_^DJI_ret', 'Volume_^GSPC_ret', 'Volume_^IXIC_ret']
[INFO] 병합 후 데이터 shape: (1257, 271)
Epoch 1/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 6s 52ms/step - loss: 0.1493 - val_loss: 0.0868
Epoch 2/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - loss: 0.1323 - val_loss: 0.0945
Epoch 3/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - loss: 0.1311 - val_loss: 0.0806
Epoch 4/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.1266 - val_loss: 0.0836
Epoch 5/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.1235 - val_loss: 0.0915
Epoch 6/6

In [ ]:
for agent_id, agent in debate_agent.agents.items():
    scaler = getattr(agent, "scaler", None)
    y_scaler = getattr(scaler, "y_scaler", None)
    print(f"{agent_id}:", type(y_scaler), y_scaler)
    print(agent.ticker)
    # if agent_id =='MacroSentiAgent':
    #     print(agent.macro_df.columns[:20])


In [3]:
opinions = debate_agent.get_opinion(0, "NVDA")

[TechnicalAgent] searcher 실행
⚙️ NVDA TechnicalAgent rebuild requested. Building dataset...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[TechnicalAgent] X_seq: (981, 55, 13), y_seq: (981, 1)
✅ NVDA TechnicalAgent dataset saved to CSV (981 samples, 13 features)
[MacroSentiAgent] X_seq: (1221, 14, 13), y_seq: (1221, 1)
✅ NVDA MacroSentiAgent dataset saved to CSV (1221 samples, 13 features)
[SentimentalAgent] X_seq: (1195, 40, 8), y_seq: (1195, 1)
✅ NVDA SentimentalAgent dataset saved to CSV (1195 samples, 8 features)
■ TechnicalAgent StockData 생성 완료 (NVDA, USD)
[TechnicalAgent] pretrain 실행
[10:17:45] Pretraining TechnicalAgent
  Epoch 005 | Loss: 0.388069
  Epoch 010 | Loss: 0.384832
  Epoch 015 | Loss: 0.389380
  Epoch 020 | Loss: 0.373048
  Epoch 025 | Loss: 0.403346
  Epoch 030 | Loss: 0.382754
  Epoch 035 | Loss: 0.372037
  Epoch 040 | Loss: 0.393725
  Epoch 045 | Loss: 0.394798
 TechnicalAgent 모델 학습 및 저장 완료: models\NVDA_TechnicalAgent.pt
[TechnicalAgent] predict 실행
[TechnicalAgent] reviewer_draft 실행
■ TechnicalAgent StockData 생성 완료 (NVDA, USD)
  - TechnicalAgent: next_close=199.6954
MacroSentiAgent의 데이터 로드.. macro_s

[*********************100%***********************]  15 of 15 completed


[MacroSentimentAgent] Macro data: (43, 91)
   ↳ Downloading NVDA ...


...

[MacroSentimentAgent] Stock data: (42, 9)
[MacroSentimentAgent] Data shape: (42, 99)
[MacroSentimentAgent] Feature engineering complete. Final shape: (42, 170)
[OK] 매크로 데이터 수집 완료: (42, 171)
[INFO] 피처 정리 및 스케일링 중...
[Check] 입력 피처 수: 169 / 스케일러 기준 피처 수: 169
[OK] 스케일링 및 시퀀스 변환 완료
■ macro_sercher StockData 생성 완료 (NVDA)
MacroSentiAgent의 예측
[INFO] 예측 수행 중...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step
NVDA: 마지막 종가=198.95 → 예측 종가=203.34 (예상 수익률 2.21%)

================= 예측 결과 (표) =================
  Ticker  Last_Close  Predicted_Close  Predicted_Return  Predicted_%  \
0   NVDA  198.951001       203.343635          0.022079     2.207898   

   uncertainty  confidence  
0     0.057162   17.494215  

================= 예측 결과 (값) =================
{'NVDA': 203.3436353592868}
[MacroSentiAgent] LLM (macro_reviewer_draft)
1️⃣ Collecting macro features (15 tickers)...


[*********************100%***********************]  15 of 15 completed


[MacroSentimentAgent] Macro data: (43, 91)
   ↳ Downloading NVDA ...


[*********************100%***********************]  1 of 1 completed


[MacroSentimentAgent] Stock data: (42, 9)
[MacroSentimentAgent] Data shape: (42, 99)
[MacroSentimentAgent] Feature engineering complete. Final shape: (42, 170)

3️⃣ Running GradientAnalyzer for interpretability...
[INFO] Running Gradient × Input + Integrated Gradients analysis...
[DEBUG] Gradient × Input output shape: (1, 42, 169)
[DEBUG] x_input shape before IG: (1, 42, 169)
[DEBUG] interpolated shape: (51, 42, 169)
[INFO] IG–G×I correlation (agreement_ratio): 0.8232
[INFO] Gradient analysis completed successfully.

4️⃣ Generating explanation using LLM...

================= pred_prices:{'NVDA': 203.3436353592868} =================

================= LLM Explanation =================
(1) Feature Trend 분석: 최근 시점에서 'High_GC=F', 'Adj Close_GC=F', 'Low_GC=F' 등 금 선물 관련 변수들의 영향력이 커졌으며, 이는 금 가격 변동성이 시장 불확실성 증가와 연동되어 나타난 결과로 보입니다. 반면, QQQ 관련 변수들의 영향력은 상대적으로 약화되었는데, 이는 기술주 중심의 시장 변동성 완화와 연관될 수 있습니다. 이러한 변화는 금 관련 변수들이 NVDA 주가 예측에 점차 중요한 신호로 작용함을 시사합니다.

(2) Model Consistency 분석: Integrated Gradi

In [4]:
print(f"opinions:{opinions}")

opinions:{'TechnicalAgent': Opinion(agent_id='TechnicalAgent', target=Target(next_close=199.6954, uncertainty=0.0033, confidence=0.9967, , reason='1) vol_20d/중심 시점 (0.2532×0.0609=0.0154): 20일 거래량은 최근 거래량 변화를 나타내며, 거래 활발함을 의미합니다. 2) obv/중심 시점 (0.2171×0.0609=0.0132): 누적거래량지표로 매수세 강도를 반영합니다. 3) log_ret_lag1/중심 시점 (0.1563×0.0609=0.0095): 전일 수익률은 단기 가격 변동성을 보여줍니다. 4) ret_3d/중심 시점 (0.1498×0.0609=0.0091): 3일 수익률은 중기 추세를 나타냅니다. 2025-06-12~2025-06-14 기간 동안 거래량과 매수세가 집중되며, 가격 상승에 긍정적 영향을 미쳤습니다. vol_20d와 obv가 높은 영향 점수로 거래량과 매수세가 강하게 작용했음을 알 수 있고, log_ret_lag1과 ret_3d도 상승 추세를 뒷받침합니다. 반면, 일부 단기 변동성 지표의 영향 점수는 낮아 과열 신호나 되돌림 가능성은 제한적이었습니다(예: macd 영향 점수 0.0046×0.0609=0.0003). 종합하면, 거래량과 매수세가 견고하고 단기 및 중기 상승 흐름이 유지되어 다음날 종가 상승 예측이 사용자 입장에서 신뢰할 만합니다.'), 'MacroSentiAgent': Opinion(agent_id='MacroSentiAgent', target=Target(next_close=203.3436353592868, uncertainty=0.05716174468398094, confidence=17.49421501159668), reason='1. 기술적 분석 측면에서, 주요 기술 지표인 QQQ의 종가(Close_QQQ)와 금(GC=F)의 고가(High_GC=F), 저가(Low_GC=F),

In [5]:
print(f"opinions:{opinions.get('MacroSentiAgent')}")

opinions:Opinion(agent_id='MacroSentiAgent', target=Target(next_close=203.3436353592868, uncertainty=0.05716174468398094, confidence=17.49421501159668), reason='1. 기술적 분석 측면에서, 주요 기술 지표인 QQQ의 종가(Close_QQQ)와 금(GC=F)의 고가(High_GC=F), 저가(Low_GC=F), 조정종가(Adj Close_GC=F)가 모두 긍정적인 신호를 보이고 있다. 특히 QQQ의 종가는 최근 상승세를 유지하며 거래량도 안정적이어서 단기 모멘텀이 강한 상태임을 시사한다. 2. 거시경제 요인으로는 금(GC=F) 가격이 상승하는 가운데, 이는 인플레이션 우려와 안전자산 선호 심리를 반영한다. 또한, 환율 변동성은 상대적으로 낮아 유동성 환경이 안정적이며, 금리 지표(^FVX, ^TNX 등)의 영향은 제한적이나, 정책 방향은 완화적 기조를 유지하는 것으로 판단된다. 3. Gradient 기반 해석에서는 Integrated Gradients와 Gradient × Input 모두 금(GC=F)의 고가, 조정종가가 가장 큰 영향력을 보이며, QQQ 종가도 중요 변수로 일관되게 나타났다. 두 기법 간의 일관성(agreement ratio 0.82)이 높아 변수 중요도 해석에 신뢰를 더한다. 다만 일부 환율 및 변동성 지표는 순위 차이가 커 불확실성이 존재한다. 4. 민감도 및 안정성 분석 결과, 금(GC=F)의 저가와 고가, 조정종가가 높은 민감도와 안정성을 보여 구조적 트렌드 요인으로 작용한다. 반면, 나스닥 거래량(Volume_^IXIC)은 민감도는 높으나 변동성이 있어 단기 리스크 요인으로 평가된다. 5. 종합적으로, 기술적 모멘텀과 거시경제 환경이 긍정적이며, 금 가격 상승이 인플레이션 헤지 및 안전자산 선호를 반영해 시장 심리를 지지한다. Gradient 해석 결과도 이를 뒷받침하여 다음 거래일 종가는 소폭 상승할 가능성이

리뷰탈 해보기..

In [5]:
debate_agent.get_rebuttal(1)

ValueError: TechnicalAgent 데이터 구조 오류: dict형 컬럼 데이터가 필요함

get_revise 해보기

In [5]:
debate_agent.get_revise(1)


[INFO] 피처 정리 및 스케일링 중...
[Check] 입력 피처 수: 169 / 스케일러 기준 피처 수: 169
[OK] 스케일링 및 시퀀스 변환 완료
[MacroSentiAgent] fine-tuning 완료: loss=0.037135
[INFO] 피처 정리 및 스케일링 중...
[Check] 입력 피처 수: 169 / 스케일러 기준 피처 수: 169
[OK] 스케일링 및 시퀀스 변환 완료
[INFO] 예측 수행 중...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step
NVDA: 마지막 종가=198.20 → 예측 종가=204.78 (예상 수익률 3.32%)

================= 예측 결과 (표) =================
  Ticker  Last_Close  Predicted_Close  Predicted_Return  Predicted_%  \
0   NVDA  198.195001       204.783759          0.033244     3.324381   

   uncertainty  confidence  
0     0.078282   12.774376  

================= 예측 결과 (값) =================
{'NVDA': 204.78375870363166}
[MacroSentiAgent] revise 완료 → new_close=204.78, loss=0.03713538125157356
[warn] Non-numeric features dropped in load_dataset(): ['date']
■ SentimentalAgent StockData 생성 완료 (NVDA, USD)


C:\Users\SPOTV\IdeaProjects\capstone_project\agents\base_agent.py:439: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor = torch.tensor(X_input, dtype=torch.float32).to(device)


[SentimentalAgent] fine-tuning 완료: loss=0.000000
[warn] Non-numeric features dropped in load_dataset(): ['date']
■ SentimentalAgent StockData 생성 완료 (NVDA, USD)
[SentimentalAgent] revise 완료 → new_close=114.90, loss=0.0
 Round 1 revise 완료 및 opinions 갱신 (2 agents)


{'MacroSentiAgent': Opinion(agent_id='MacroSentiAgent', target=Target(next_close=204.78375870363166, uncertainty=0.0782817006111145, confidence=12.774375915527344), reason='금리와 인플레이션 관련 핵심 변수인 Close_GC=F(금 가격), High_^GSPC(S&P 500 지수), Open_^DJI(다우존스 지수)가 모델에서 높은 중요도와 안정성을 보이며, Integrated Gradients와 Gradient×Input 간 일관성도 양호하여 신뢰도가 높습니다. 최근 시계열 분석에서 금 가격과 주요 주가지수는 완만한 상승세를 유지하고 있으며, 이는 유동성 확대와 투자 심리 개선을 반영합니다. 민감도 분석 결과 금 가격 관련 변수들은 안정적인 민감도를 보여 구조적 상승 모멘텀을 지지하지만, 일부 주식시장 지수 변수는 민감도가 다소 높아 단기 변동성 위험이 존재합니다. 반면, 금리 인상 기조나 실질금리 상승, 유동성 축소 신호는 뚜렷하지 않고, 오히려 금리 안정과 인플레이션 완화 조짐이 관찰되어 거시경제 환경은 완화적입니다. Stability_summary에서 주요 변수들의 변동성은 낮아 단기 이벤트성 요인보다는 구조적 펀더멘털에 기반한 신뢰할 만한 신호로 판단됩니다. 동료 에이전트의 NVDA 주가 하락 전망은 개별 기업 실적과 산업 특성에 기인한 것으로, 본 금 가격 및 거시경제 지표와 직접적인 연관성은 제한적입니다. 따라서, 금리·인플레이션·유동성 지표의 완화적 방향성과 높은 변수 안정성, 일관성에 근거하여 기존 예측값 173.898에서 약 +2.0% 상향 조정한 177.376으로 수정한다.'),
 'SentimentalAgent': Opinion(agent_id='SentimentalAgent', target=Target(next_close=114.89917173633057, uncertainty=0.108687624335

결과 도출해보기

In [ ]:
debate_agent.get_ensemble()

통합해서 한번에 실행하기: 기본 라운드는 총 3회. 클래스 정의떄 수정 가능

In [ ]:
#debate_agent.run()